# LEXI Tutorial
Generated by [pandoc](https://pandoc.org/) from [lexi-tutorial.ipynb](lexi_tutorial.ipynb).

Author: [Ramiz Qudsi](https://www.qudsiramiz.space/)

Latest Update Date: {date_time}

This Notebook will walk you through the process of using the LEXI code with the final goal of
producing and saving x-ray data from LEXI spacecraft.


If you need to install LEXI, please see the [lexi installation
instructions](lexi_install_instructions.md) for detailed guidance.

`LEXI`, in this context, is a package developed in Python to ingest the data from the
[LEXI](https://sites.bu.edu/lexi/) spacecraft and produce x-ray data images in RA-DEC coordinate system.

## LEXI package description

`LEXI` package has the following functions:
```bash
  - `get_spc_prams`

  - `get_exposure_maps`

  - `get_sky_backgrounds`

  - `get_lexi_images`
```
Details of each function are described in the following sections.

### `get_spc_prams`

- `get_spc_prams`: to get the spacecraft's ephemeris data. It takes the following inputs:
- Required:
    ```bash
        - `time_range`: a list of two elements, the start and end time of the observation in UTC. The format of the time can be any of the following:
            - string: `YYYY-MM-DD HH:MM:SS`
            - float: Unix time or the number of seconds since 1970-01-01 00:00:00 UTC
            - datetime object: `datetime.datetime(YYYY, MM, DD, HH, MM, SS)`
        - Optional:

        - `time_zone`: the time zone of the input time. The default value is `UTC`.

        - `time_step`: the time step in seconds at which the user desires to have look-direction computed. The default value is set to 5 seconds and is sufficient for most of the cases. The user can change it to a different value if needed.

        - `interp_method`: the method of interpolation. The default value is `linear`. The user can change it to `cubic` or something similarif needed.

        - `verbose`: a boolean value to print the progress of the function. The default value is `True`.
    ```

The function returns the interpolated ephemeris data in a pandas dataframe.
- Output:
    ```bash
        - A pandas dataframe with the following columns:
            - `epoch_utc`: Time in UTC
            - `ra`: Right Ascension in degrees
            - `dec`: Declination in degrees
            - `roll`: Roll angle in degrees
    ```

The interpolation is done internally on the ephemeris data using the `time_step` parameter.
`time_step` is the time step in seconds at which the user desires to have look-direction computed.
The default value is set to 5 seconds and is sufficient for most of the cases. The user can change it
to a different value if needed.

### `get_exposure_maps`

- `get_exposure_maps`: The function to compute the exposure map for any given time interval. It
takes the following inputs:

- Required:
    ```bash
    - `time_range`: a list of two elements, the start and end time of the observation in UTC. The format of the time can be any of the following:
            - string: YYYY-MM-DD HH:MM:SS
            - float: Unix time or the number of seconds since 1970-01-01 00:00:00 UTC
            - datetime object: datetime.datetime(YYYY, MM, DD, HH, MM, SS)
    ``` 
Using the `time_range` parameter, the function will call `get_spc_prams` internally to get the ephemeris data.

- Optional:
    ```bash

    - `ra_range`: a list of two elements, the start and end RA of over which the user want the exposure maps to be computed. If not provided, the function computes the exposure map over     the entire range of possible RA values (0 to 360 degrees).

    - `dec_range`: a list of two elements, the start and end Dec of over which the user want the     exposure maps to be computed. If not provided, the function computes the exposure map over the entire range of possible Dec values (-90 to 90 degrees).

    - `ra_res`: the resolution of the RA bins in degrees. The default value is set to 0.1 degrees. The user can change it to a different value if needed.

    - `dec_res`: the resolution of the Dec bins in degrees. The default value is set to 0.1 degrees.

    - `nbins`: The number of bins to be used while computing the exposure map. It can be a single integer or a list of two integers. If a single integer is provided, the function will use the same number of bins for both RA and Dec. If a list of two integers is provided, the first element will be used for RA and the second element will be used for Dec. Note that if `ra_res` and `dec_res` are provided, the function will use the number of bins computed from the resolution values. However, if either of them is not provided, the function will use the number of bins provided by the user.

    - `time_step`: time step in seconds at which the user desires to have look-direction computed. The default value is set to 5 seconds and is sufficient for most of the cases. The user can change it to a different value if needed.

    - `time_integrate`: The integration time in seconds. This the length of time for wwhich each exposure map is computed. The default value is set to 600 seconds 10 minutes. If it is not provided by the user, the function will assume the time difference between the start and end time of the observation as the integration time. For example, if the provided `time_range` is ['2020-01-01 00:00:00', '2020-01-01 02:10:00'], the function will assume the integration time to be 7800 seconds (2 hours and 10 minutes). However, if the user provides a different `integration_time`, let us say 600 seconds, the function will compute the exposure maps for 10 minutes each, there by producing 22 exposure maps.

    - `save_exposure_maps`: a boolean value to indicate whether the user wants to save the exposure maps as a PNG files. The default value is set to `False`. If the user wants to save the exposure maps, the function will save them in the a folder named `figures/exposure_maps/` the current working directory. The function will also create a `.npy` file containing the the in a folder named `data/exposure_maps/` in the current working directory. The name of the  `.npy` file will be `lexi_exposure_maps_Tstart_[YYYYMMDD_HHMMSS]_Tstop_[YYYYMMDD_HHMMSS]_RAstart_[RAstart]_RAstop_[RAstop]_Decstart_[Decstart]_Decstop_[Decstop]_RAres_[RAres]_Decres_[Decres]_tint_[tint].npy`.
    ```

In the namefile, everything witihn the square brackets will be replaced by the actual values as computed by the function. For example, if the user provides the following inputs:

```bash
    - `time_range`: ['2020-01-01 00:00:00', '2020-01-01 02:10:00']
    - `ra_range`: [0, 360]
    - `dec_range`: [-90, 90]
    - `ra_res`: 0.1
    - `dec_res`: 0.1
    - `nbins`: [3600, 1800]
    - `time_step`: 5
    - `time_integrate`: 600
```

The function will save the exposure maps as:
```bash
    `lexi_exposure_maps_Tstart_20200101_000000_Tstop_20200101_021000_RAstart_0_RAstop_360_Decstart_-90_Decstop_90_RAres_0.1_Decres_0.1_tint_600.npy`
```
in the `data/exposure_maps/` folder and the PNG files in the `figures/exposure_maps/` folder.

The function returns a dictionary containing the following keys:
```bash
    - `exposure_maps` : numpy array, Exposure maps
    - `ra_arr` : numpy array, Right ascension array
    - `dec_arr` : numpy array, Declination array
    - `time_range` : list
        Time range of the exposure maps
    - `time_integrate` : int or float, Integration time in seconds of the exposure maps
    - `ra_range` : list, Right ascension range of the exposure maps in degrees
    - `dec_range` : list, Declination range of the exposure maps in degrees
    - `ra_res` : float, Right ascension resolution of the exposure maps in degrees
    - `dec_res` : float, Declination resolution of the exposure maps in degrees
    - `start_time_arr` : numpy array, Start time of each exposure map
    - `stop_time_arr` : numpy array, Stop time of each exposure map
```

### `get_sky_backgrounds`

- `get_sky_backgrounds`: The function to compute the sky backgrounds for any given time interval
using the ROSAT data.
It takes the following inputs:
- Required:

    ```bash
        - `time_range`: a list of two elements, the start and end time of the observation in UTC. The format
        of the time can be any of the following:
        - string: `YYYY-MM-DD HH:MM:SS`
        - float: Unix time or the number of seconds since 1970-01-01 00:00:00 UTC
        - datetime object: `datetime.datetime(YYYY, MM, DD, HH, MM, SS)`
    ```
Using the `time_range` parameter, the function will call `get_spc_prams` internally to get the
ephemeris data. The function will also call `get_exposure_maps` internally to get the exposure maps.

- Optional:
    ```bash
        - `ra_range`: a list of two elements, the start and end RA of over which the user want the sky backgrounds to be computed. If not provided, the function computes the sky backgrounds over the entire range of possible RA values (0 to 360 degrees).

        - `dec_range`: a list of two elements, the start and end Dec of over which the user want the sky backgrounds to be computed. If not provided, the function computes the sky backgrounds over the entire range of possible Dec values (-90 to 90 degrees).
        
        - `ra_res`: the resolution of the RA bins in degrees. The default value is set to 0.1 degrees. The user can change it to a different value if needed.
        
        - `dec_res`: the resolution of the Dec bins in degrees. The default value is set to 0.1 degrees.
        
        - `nbins`: The number of bins to be used while computing the sky backgrounds. It can be a single integer or a list of two integers. If a single integer is provided, the function will use the same number of bins for both RA and Dec. If a list of two integers is provided, the first element will be used for RA and the second element will be used for Dec. Note that if `ra_res` and `dec_res` are provided, the function will use the number of bins computed from the resolution values. However, if either of them is not provided, the function will use the  number of bins provided by the user.

        - `time_step`: time step in seconds at which the user desires to have look-direction computed. The default value is set to 5 seconds and is sufficient for most of the cases. The user can change it to a different value if needed.

        - `time_integrate`: the integration time in seconds. This the length of time for wwhich each exposure map is computed. The default value is set to 600 seconds (10 minutes). If it is not provided by the user, the function will assume the time difference between the start and end time of the observation as the integration time. For example, if the provided `time_rangenge` is ['2020-01-01 00:00:00', '2020-01-01 02:10:00'], the function will assume the integration time to be 7800 seconds (2 hours and 10 minutes). However, if the user provides a different  `integration_time`, let us say 600 seconds, the function will compute the exposure maps for 10 minutes each, there by producing 22 sky background images.

        - `save_sky_backgrounds`: a boolean value to indicate whether the user wants to save the sky background images as a PNG files. The default value is set to `False`. If the user wants to save the sky background images, the function will save them in the a folder named `figures/sky_backgrounds/` in the current working directory.
    ```

The function returns a dictionary containing the following keys:
```bash
    - `sky_backgrounds` : numpy array, Exposure maps
    - `ra_arr` : numpy array, Right ascension array
    - `dec_arr` : numpy array, Declination array
    - `time_range` : list
        Time range of the exposure maps
    - `time_integrate` : int or float, Integration time in seconds of the exposure maps
    - `ra_range` : list, Right ascension range of the exposure maps in degrees
    - `dec_range` : list, Declination range of the exposure maps in degrees
    - `ra_res` : float, Right ascension resolution of the exposure maps in degrees
    - `dec_res` : float, Declination resolution of the exposure maps in degrees
    - `start_time_arr` : numpy array, Start time of each exposure map
    - `stop_time_arr` : numpy array, Stop time of each exposure map
```

### `get_lexi_images`

- `get_lexi_images`: The function to compute the background corrected or uncorrected x-ray image
from LEXI data. The function takes the following inputs:

- Required:
    ```bash
        - `time_range`: a list of two elements, the start and end time of the observation in UTC. The format of the time can be any of the following:
            - string: `YYYY-MM-DD HH:MM:SS`
            - float: Unix time or the number of seconds since 1970-01-01 00:00:00 UTC
            - datetime object: `datetime.datetime(YYYY, MM, DD, HH, MM, SS)`
    ```
    Using the `time_range` parameter, the function will call `get_spc_prams` internally to get the ephemeris data. The function will also call `get_exposure_maps` internally to get the exposure maps. The function will also call `get_sky_backgrounds` internally to get the sky backgrounds.

- Optional:
    ```bash
        - `background_correction_on`: a boolean value to indicate whether the user wants to apply the background correction to the x-ray image. The default value is set to `True`. If the user wants to apply the background correction, the function will subtract the sky backgrounds from the x-ray image.
        
        - `ra_range`: a list of two elements, the start and end RA of over which the user want the
        sky backgrounds to be computed. If not provided, the function computes the sky backgrounds
        over the entire range of possible RA values (0 to 360 degrees).
        
        - `dec_range`: a list of two elements, the start and end Dec of over which the user want the
        sky backgrounds to be computed. If not provided, the function computes the sky backgrounds
        over the entire range of possible Dec values (-90 to 90 degrees).
        
        - `ra_res`: the resolution of the RA bins in degrees. The default value is set to 0.1 degrees.
        The user can change it to a different value if needed.
        
        - `dec_res`: the resolution of the Dec bins in degrees. The default value is set to 0.1
        degrees.

        - `nbins`: The number of bins to be used while computing the sky backgrounds. It can be a single integer or a list of two integers. If a single integer is provided, the function will use the same number of bins for both RA and Dec. If a list of two integers is provided, the first element will be used for RA and the second element will be used for Dec. Note that if `ra_res` and `dec_res` are provided, the function will use the number of bins computed from the resolution values. However, if either of them is not provided, the function will use the  number of bins provided by the user.

        - `time_step`: time step in seconds at which the user desires to have look-direction computed. The default value is set to 5 seconds and is sufficient for most of the cases. The user can change it to a different value if needed.

        - `time_integrate`: the integration time in seconds. This the length of time for wwhich each exposure map is computed. The default value is set to 600 seconds (10 minutes). If it is not provided by the user, the function will assume the time difference between the start and end time of the observation as the integration time. For example, if the provided `time_range` is ['2020-01-01 00:00:00', '2020-01-01 02:10:00'], the function will assume the integration time to be 7800 seconds (2 hours and 10 minutes). However, if the user provides a different

        `integration_time`, let's say 600 seconds, the function will compute the exposure maps for 10
        minutes each, there by producing 22 sky background images.
    ```

The function returns a dictionary containing the following keys:
```bash
    - `lexi_images` : numpy array, Lexi images as computed by the function numpy array
    - `ra_arr` : numpy array, Right ascension array
    - `dec_arr` : numpy array, Declination array
    - `time_range` : list
        Time range of the exposure maps
    - `time_integrate` : int or float, Integration time in seconds of the exposure maps
    - `ra_range` : list, Right ascension range of the exposure maps in degrees
    - `dec_range` : list, Declination range of the exposure maps in degrees
    - `ra_res` : float, Right ascension resolution of the exposure maps in degrees
    - `dec_res` : float, Declination resolution of the exposure maps in degrees
    - `start_time_arr` : numpy array, Start time of each exposure map
    - `stop_time_arr` : numpy array, Stop time of each exposure map
```

# Using the LEXI Code

## Starting with the environment

 Assuming that you have installed the LEXI package using the instructions provided in the [README.md](./../README.md) file, you can start using LEXI code by following the steps below.

- `cd` into you working directory and activate the environment
    ```bash
        cd /path/to/your/working/directory
    ```

- Activate the virtual environment
    - On Linux or MacOS
    ```bash
        source /path/to/your/lexi/env/bin/activate
    ```
    - On Windows
    ```bash
        \path\to\your\lexi\env\Scripts\activate.bat
    ```

- If you are trying to run this notebook, you will need jupyter notebook installed in your environment. If you have not installed it, you can do so by running the following command:
    ```bash
        pip install jupyter
    ```

- Once you have installed the jupyter notebook, you can start the notebook by running the following command:
    ```bash
        jupyter notebook
    ```
    This will open a new tab in your default web browser showing the contents of your working directory.

You can then open this notebook by clicking on it.


## Importing the LEXI package


In [1]:
# Import LEXI
from lexi import lexi


In [2]:
# Check if lexi was imported correctly by printing the main LEXI docstring
print(lexi.__doc__)



The lexi is a package developed using the Python programming language. The package is
designed to provide a simple list of functions to work with the LEXI dataset. The package has
following usable modules:
    - get_spc_prams: This module is used to get the spacecraft parameters from the LEXI dataset using
      a specified time range.
    - get_exposure_maps: This module is used to get the exposure maps from the LEXI dataset using a
      specified time range and some other input parameters.
    - get_sky_backgrounds: This module is used to get the sky backgrounds from the LEXI dataset which
      corresponds to the exposure maps. The module uses the exposure maps to get the sky backgrounds.
    - get_lexi_images: This module is used to get the LEXI images from the LEXI dataset using a
      specified time range and some other input parameters. The module uses the exposure maps and sky
      backgrounds to get the LEXI images. One can either get a background corrected image or a raw


In [3]:
# Get the spacecraft data from the LEXI database
spacecraft_data = lexi.get_spc_prams(time_range=["2024-07-08T21:43:41", "2024-07-08T21:47:41"])


Timezone set to  UTC  



In [4]:
print(spacecraft_data.head())


                              epoch_mjd    earth_ra  earth_dec      sun_ra  \
epoch_utc                                                                    
2024-07-08 21:43:00+00:00  60499.905290  324.947547 -18.051252  108.191896   
2024-07-08 21:43:05+00:00  60499.905348  324.948239 -18.050981  108.191954   
2024-07-08 21:43:10+00:00  60499.905405  324.948931 -18.050710  108.192011   
2024-07-08 21:43:15+00:00  60499.905462  324.949623 -18.050439  108.192069   
2024-07-08 21:43:20+00:00  60499.905520  324.950315 -18.050168  108.192127   

                             sun_dec      sco_ra    sco_dec       mp_az  \
epoch_utc                                                                 
2024-07-08 21:43:00+00:00  22.372867  244.982213 -15.640577  248.463063   
2024-07-08 21:43:05+00:00  22.372860  244.982213 -15.640577  248.463155   
2024-07-08 21:43:10+00:00  22.372853  244.982213 -15.640577  248.463248   
2024-07-08 21:43:15+00:00  22.372847  244.982213 -15.640577  248.463340   
202

In [5]:
# Print the keys of the spacecraft data
print(spacecraft_data.keys())


Index(['epoch_mjd', 'earth_ra', 'earth_dec', 'sun_ra', 'sun_dec', 'sco_ra',
       'sco_dec', 'mp_az', 'mp_el', 'mp_ra', 'mp_dec'],
      dtype='object')


In [7]:
# Get the exposure map
exposure_map = lexi.get_exposure_maps(time_range=["2024-07-08T21:43:41", "2024-07-08T21:47:41"])


Timezone set to  UTC  

 Integration time  (time_integrate)  not provided. Setting integration time to the time span of the spacecraft ephemeris data:  240.0 seconds 

Exposure map not found, computing now. This may take a while 



In [8]:
# Get the background data
sky_backgrounds_dict = lexi.get_sky_backgrounds(time_range=["2024-07-08T21:43:41", "2024-07-08T21:47:41"])


Timezone set to  UTC  

 Integration time  (time_integrate)  not provided. Setting integration time to the time span of the spacecraft ephemeris data:  240.0 seconds 

Exposure map not found, computing now. This may take a while 

Sky background not found, computing now. This may take a while 

